## Performing Real-Time Test

In [13]:
import cv2
import numpy as np
import mediapipe as mp
import pickle
import matplotlib.pyplot as plt

In [ ]:
# !jupyter nbconvert --to script Create_dataset.ipynb

[NbConvertApp] Converting notebook Create_dataset.ipynb to script
[NbConvertApp] Writing 3299 bytes to Create_dataset.py


In [15]:
from Extracting_Landmarks import extract_keypoints , mp_holistic , mp_drawing , mediapipe_detection , drawing_styled_landmarks
from Create_dataset import actions

In [ ]:
# File Path to store the landmark data

DATA_PATH = os.path.join("LANDMARK_DATA")

# Defining the actions 

actions = np.array(['hello' , 'thankyou' , 'iloveyou' , 'sorry' ,'A' , 'B' , 'C' , 'D' , 'E' , 'F'])
# Defining the number of videos per action

no_of_videos = 100

# Defining the number of frames per video

no_of_frames = 30

In [16]:
# actions = np.array(['hello' , 'thankyou' , 'iloveyou'])

Importing the model

In [17]:
with open('LRCN_model' , 'rb') as file:
 
 model = pickle.load(file)

In [18]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 30, 1662, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 831, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 15, 831, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 15, 831, 32)    │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 415, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 7, 415, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 7, 415, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 7, 26560)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │     3,403,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,281,899 (39.22 MB)

 Trainable params: 3,427,299 (13.07 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,854,600 (26.15 MB)

Displaying probability for words

In [19]:
colors = [(245 , 117 , 16) , (117 , 245 , 16) , (16 , 117 , 245)]

def prob_visualization(res , actions , input_frame , colors):
  
  output_frame = input_frame.copy()
  for num , prob in enumerate(res):
    cv2.rectangle(output_frame ,(0 , 60 + num*40) , (int(prob*100), 90+num*40) , colors[num] , -1)
    cv2.putText(output_frame , actions[num] , (0, 85 + num*40) , cv2.FONT_HERSHEY_SIMPLEX , 1 , (255 , 255 , 255) , 2 , cv2.LINE_AA)
  
  return output_frame

In [20]:
# # Checking video camera availablity 

# sequence = []
# sentence = []
# threshold = 0.9
# res = None
# predictions = []
 
# cap = cv2.VideoCapture(0)  # accesses the webcam

# with mp_holistic.Holistic(min_detection_confidence= 0.7 , min_tracking_confidence= 0.7) as holistic:
#    while cap.isOpened():  #This loop continiously read frames from the camera cap

#       # Reading the frames
#       ret , frame = cap.read() # ret is a boolean variable that is True is a frame is being read

#       # Using Detection Function

#       image , landmark_results = mediapipe_detection(frame , holistic )
#       print(landmark_results)

#       #Drawing Landmarks onto the frames on the screen
#       drawing_styled_landmarks(image , landmark_results)

#       # Adding prediction logic
#       keypoints = extract_keypoints(landmark_results)
#       sequence.append(keypoints)
#       sequence = sequence[-30:]
   

#       if len(sequence) == 30:
#          res = model.predict(np.expand_dims(np.expand_dims(sequence , axis = -1) , axis = 0 ))[0]
         
#          predictions.append(np.argmax(res))

#       # Visualization logic
      
#       if len(predictions) >= 10 and np.unique(predictions[-10:])[0] == np.argmax(res):
#          if res is not None and res[np.argmax(res)] > threshold:

#             if len(sentence) > 0:

#                if actions[np.argmax(res)] != sentence[-1]:
#                   sentence.append(actions[np.argmax(res)])
            
#             else: 

#                sentence.append(actions[np.argmax(res)])
      
#       if len(sentence) > 5:
#          sentence = sentence[-5:]
      
#       #Prob Viz
#       if res is not None:
#        image = prob_visualization(res , actions , image , colors)

#       cv2.rectangle(image, (0, 0), (image.shape[1], 40), (245, 117, 16), -1)
#       cv2.putText(image , ' '.join(sentence) , (3,30) , cv2.FONT_HERSHEY_SIMPLEX , 1 , (255 , 255 , 255) , 2 , cv2.LINE_AA)

#       # Displaying frames to screen 
#       cv2.imshow('OpenCV' , image) 

#       if cv2.waitKey(10) & 0xFF == ord('q'):
#          break
      
#    cap.release()
#    cv2.destroyAllWindows()

In [21]:
# Checking video camera availablity 

sequence = []
sentence = []
threshold = 0.7
res = None
predictions = []
 
cap = cv2.VideoCapture(0)  # accesses the webcam

with mp_holistic.Holistic(min_detection_confidence= 0.7 , min_tracking_confidence= 0.7) as holistic:
   while cap.isOpened():  #This loop continiously read frames from the camera cap

      # Reading the frames
      ret , frame = cap.read() # ret is a boolean variable that is True is a frame is being read

      # Using Detection Function

      image , landmark_results = mediapipe_detection(frame , holistic )
      print(landmark_results)

      #Drawing Landmarks onto the frames on the screen
      drawing_styled_landmarks(image , landmark_results)

      # Adding prediction logic
      keypoints = extract_keypoints(landmark_results)
      sequence.append(keypoints)
      sequence = sequence[-30:]
   

      if len(sequence) == 30:
         res = model.predict(np.expand_dims(sequence , axis = 0 ))[0]
         predictions.append(np.argmax(res))

      # Visualization logic
      
      if len(predictions) >= 20 and np.unique(predictions[-20:])[0] == np.argmax(res):
         if res is not None and res[np.argmax(res)] > threshold:

            if len(sentence) > 0:

               if actions[np.argmax(res)] != sentence[-1]:
                  sentence.append(actions[np.argmax(res)])
            
            else: 

               sentence.append(actions[np.argmax(res)])
      
      if len(sentence) > 5:
         sentence = sentence[-4:]
      
      #Prob Viz
      if res is not None:
       image = prob_visualization(res , actions , image , colors)

      cv2.rectangle(image, (0, 0), (image.shape[1], 40), (245, 117, 16), -1)
      cv2.putText(image , ' '.join(sentence) , (3,30) , cv2.FONT_HERSHEY_SIMPLEX , 1 , (255 , 255 , 255) , 2 , cv2.LINE_AA)

      # Displaying frames to screen 
      cv2.imshow('OpenCV' , image) 

      if cv2.waitKey(10) & 0xFF == ord('q'):
         break
      
   cap.release()
   cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti